In [1]:
disambig = {}
disambig['similarity'] = 'similarity measure'
disambig['representation'] = 'feature learning'
disambig['analytical'] = 'Analytics'
disambig['cluster'] = 'computer cluster'
disambig['precision'] = 'precision statistics'
disambig['spark'] = 'Apache spark'
disambig['c#'] = 'c sharp language'
disambig['pca'] = 'principal component analysis'
disambig['csharp'] = 'c sharp language'
disambig['modeling'] = 'statistical model'
disambig['regression'] = 'regression analysis'
disambig['metrics'] = 'precision recall'
disambig['python'] = 'python language'
disambig['mr'] = 'map reduce'
disambig['recall'] = 'recall statistics'
disambig['application'] = 'application software'
disambig['developer'] = 'software developer'
disambig['ica'] = 'ica statistics'
disambig['c-sharp'] = 'c sharp language'
disambig['etl'] = 'extract transform load'
disambig['lda'] = 'lda algorithm'
disambig['transform'] = 'data transformation'
disambig['react'] = 'react javascript'
disambig['graphx'] = 'apache spark graphx'
disambig['load'] = 'extract transform load'
disambig['python3'] = 'python language'
disambig['graph'] = 'graph abstract data'
disambig['matrix'] = 'data matrix'
disambig['features'] = 'features machine learning'
disambig['vector'] = 'vector representation'
disambig['clustering'] = 'clustering algorithm'
disambig['ml'] = 'machine learning'
disambig['nlp'] = 'natural language processing'
disambig['pgm'] = 'probabilistic graphical model'

page_error_lookup = {}
page_error_lookup['apachespark'] = 'apache spark'
page_error_lookup['spark-stream'] = 'spark streaming'

In [5]:
import wikipedia
import pandas as pd
import wikipedia
from nltk.tokenize import word_tokenize,sent_tokenize,MWETokenizer
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.tokenize import MWETokenizer
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
import sys
from optparse import OptionParser
from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np
import csv

#Reads from input file
df=pd.read_csv('keywords.csv',header=None)

tokenizer = MWETokenizer()
#Multi word extracter
#Tokenizing based on '-' as the input corpus has multiple words 
#concatenated with '-'
for index, row in df.iterrows():
    if row[0].find('-')!=-1:
        tokenizer.add_mwe(tuple(row[0].split('-')))
text = []
not_found = []
page_error = []
#Fetching data from wikipedia
for index, row in df.iterrows():
    try:
        content = wikipedia.page(row[0]).summary
    except wikipedia.exceptions.DisambiguationError as e:
        if row[0] not in disambig:
            not_found.append(row[0])
        else:
            try:
                content = wikipedia.page(disambig[row[0]]).summary
            except wikipedia.exceptions.DisambiguationError as e:
                not_found.append(row[0])
            except wikipedia.exceptions.PageError as e1:
                page_error.append(row[0])
    except wikipedia.exceptions.PageError as e1:
        if row[0] not in page_error_lookup:
            page_error.append(row[0])
        else:
            try:
                content = wikipedia.page(page_error_lookup[row[0]]).summary
            except wikipedia.exceptions.PageError as e2:
                page_error.append(row[0])
    sentence =  sent_tokenize(content)
    word_list = []

    for sent in sentence:
        word_list.append(tokenizer.tokenize(sent.split()))
    stops = list(stopwords.words("english"))

    #stop words
    stop_words_with_punctuation = stopwords.words("english")+ list(string.punctuation)
    filtered_words = []
    for words_list in word_list:
        temp = []
        for word in words_list:
            if word.lower() not in stop_words_with_punctuation:
                temp.append(word)
        filtered_words.append(temp)
    text.append([row[0],list(itertools.chain.from_iterable(filtered_words))])
    
data = text 


# options for the vectorizing method
op = OptionParser()
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=500,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")

def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)

sentences = []
for i in range(0,len(data)):
    sentences.append(' '.join(data[i][1]))

keys = []
for i in range(0,len(data)):
    keys.append(''.join(data[i][0]))

#vectorizing the data
hasher = HashingVectorizer(n_features=opts.n_features,
                                   stop_words='english', alternate_sign=False,
                                   norm=None, binary=False)
vectorizer = make_pipeline(hasher, TfidfTransformer())
X = vectorizer.fit_transform(sentences)

# Applying KMeans algorithm
km = KMeans(n_clusters=20, init='k-means++', max_iter=1000, n_init=1,
                verbose=False)
km.fit(X)

#persisting results back to csv
key_tuple = {}
for j in range(0,len(km.labels_)):
    for i in range(0,len(data)):
        if i==j:
            if km.labels_[j] not in key_tuple:
                key_tuple[km.labels_[j]] = [keys[i]]
            else:
                key_tuple[km.labels_[j]].append(keys[i])
with open('results.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in key_tuple.items():
        writer.writerow((value))

/home/naveen/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/naveen/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
